In [ ]:
import numpy as np
import pandas as pd
from astropy import units as u
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from astropy.coordinates import SkyCoord

In [ ]:
# Constants (SI units)
G = 6.67384e-11         # gravitational constant (m^3/kg/s^2)
c = 299792458           # speed of light (m/s)
MSUN = 1.989e30         # solar mass (kg)
PC = 3.0856775815e16    # parsec (m)

# Read in raw gbfisher data
cat_columns = ['f', 'colat', 'lon', 'A', 'incl', 'psi', 'phi', 'fdot', 'fddot',
               's_df/f', 's_colat', 's_lon', 's_dA/A', 's_incl', 's_psi',
               's_phi', 's_dfdot/fdot', 's_dfddot/fddot', 's_Omega', 'SNR']
cat = pd.read_csv('SigmasAE.dat', delimiter=' ', header=None,
                  index_col=False, names=cat_columns)
cat = cat.drop(cat[cat['fdot'].lt(0)].index, axis=0)

# Compute period in minutes
cat['P'] = 2/cat['f'] / 60

# Compute rescaled inclination in degrees
cat['deg_incl'] = 90 - np.abs(np.degrees(cat['incl']) - 90)

# Convert from ecliptic to galactic coordinates
gal = SkyCoord(cat['lon'], np.pi/2 - cat['colat'], unit=u.rad,
               frame='barycentrictrueecliptic').galactic
cat['l'] = gal.l.degree
cat['b'] = gal.b.degree

# Compute chirp mass in solar masses
cat['Mc'] = c**3/G * (5/96 * np.pi**(-8/3) * cat['f']**(-11/3)
                      * cat['fdot'])**(3/5) / MSUN

# Compute distance in parsecs
cat['D'] = 5/48*c * cat['fdot'] / (np.pi**2 * cat['f']**3 * cat['A']) / PC

# Compute "luminosity"
cat['lum'] = cat['D']**-2
cat['lum'] /= cat['lum'].sum()

# Compute KDE
kernel = gaussian_kde(cat[['l', 'b']].to_numpy().T)
cat['KDE'] = -np.log(kernel.evaluate(cat[['l', 'b']].to_numpy().T))
cat['KDE'] /= cat['KDE'].sum()

In [ ]:
# Naive weight model
ckde = 1
clum = 1
cat['Wt'] = (cat['KDE']**ckde) * (cat['lum']**clum)
cat['Wt'] /= cat['Wt'].sum()

In [ ]:
# Downsample binaries and show sky plot of downsampled binaries
P_min, P_max = 3, 10
incl_min, incl_max = 80, 90
snr_min, snr_max = 80, 179
Wt_min, Wt_max = 0, 1

cut = (
    cat['P'].between(P_min, P_max)
    & cat['deg_incl'].between(incl_min, incl_max)
    & cat['SNR'].between(snr_min, snr_max)
    & cat['Wt'].between(Wt_min, Wt_max)
)

fig = plt.figure(figsize=(15, 15))
plt.subplot(111, projection='aitoff')
gal = SkyCoord(cat[cut]['l'], cat[cut]['b'], frame='galactic', unit=u.deg)
plt.scatter(gal.l.wrap_at('180d').rad, gal.b.rad, s=10)
plt.grid()
plt.show()

cat[cut].head(n=1000)

In [ ]:
# Sky plot showing periods
fig = plt.figure(figsize=(15, 15))
plt.subplot(111, projection='aitoff')

gal = SkyCoord(cat[cut]['l'], cat[cut]['b'], frame='galactic', unit=u.deg)
plt.scatter(gal.l.wrap_at('180d').rad, gal.b.rad, s=10,
            c=cat[cut]['P'], cmap='viridis')
cbar = plt.colorbar(orientation='horizontal', pad=0.05)
cbar.set_label(label='Period (minutes)', size=15)
plt.grid()
plt.show()

# Sky plot showing distances
fig = plt.figure(figsize=(15, 15))
plt.subplot(111, projection='aitoff')
gal = SkyCoord(cat[cut]['l'], cat[cut]['b'], frame='galactic', unit=u.deg)
plt.scatter(gal.l.wrap_at('180d').rad, gal.b.rad, s=10,
            c=cat[cut]['D'], cmap='viridis')
cbar = plt.colorbar(orientation='horizontal', pad=0.05)
cbar.set_label(label='Distance (kpc)', size=15)
plt.grid()
plt.show()

# Sky plot showing inclinations
fig = plt.figure(figsize=(15, 15))
plt.subplot(111, projection='aitoff')
gal = SkyCoord(cat[cut]['l'], cat[cut]['b'], frame='galactic', unit=u.deg)
plt.scatter(gal.l.wrap_at('180d').rad, gal.b.rad, s=10,
            c=cat[cut]['deg_incl'], cmap='viridis')
cbar = plt.colorbar(orientation='horizontal', pad=0.05)
cbar.set_label(label='Inclination (degrees)', size=15)
plt.grid()
plt.show()

# Sky plot showing SNRs
fig = plt.figure(figsize=(15, 15))
plt.subplot(111, projection='aitoff')
gal = SkyCoord(cat[cut]['l'], cat[cut]['b'], frame='galactic', unit=u.deg)
plt.scatter(gal.l.wrap_at('180d').rad, gal.b.rad, s=10,
            c=cat[cut]['SNR'], cmap='viridis')
cbar = plt.colorbar(orientation='horizontal', pad=0.05)
cbar.set_label(label='SNR', size=15)
plt.grid()
plt.show()

In [ ]:
# Plot comparing population and sample marginal period distributions
fig = plt.figure()
bins = 10**np.linspace(0.1, 2, 29)
plt.hist(cat['P'], bins, histtype='bar', label='population',
         color='C1', edgecolor='black', alpha=0.5, density=True)
plt.hist(cat[cut]['P'], bins, histtype='bar', label='sample',
         color='C0', edgecolor='black', alpha=0.5, density=True)
plt.title('Marginal Period Distribution', fontsize=15)
plt.xlabel('Period (minutes)', fontsize=14)
plt.xscale('log')
plt.legend()
plt.show()

# Plot comparing population and sample marginal inclination distributions
fig = plt.figure()
bins = np.linspace(0, 90, 46)
plt.hist(cat['deg_incl'], bins, histtype='bar', label='population',
         color='C1', edgecolor='black', alpha=0.5, density=True)
plt.hist(cat[cut]['deg_incl'], bins, histtype='bar', label='sample',
         color='C0', edgecolor='black', alpha=0.5, density=True)
plt.title('Marginal Inclination Distribution', fontsize=15)
plt.xlabel('Inclination (degrees)', fontsize=14)
plt.xticks(np.linspace(0, 90, 10))
plt.legend()
plt.show()

# Plot comparing population and sample marginal distance distributions
fig = plt.figure()
bins = 10**np.linspace(-1.5, 1.4, 31)
plt.hist(cat['D']/1000, bins, histtype='bar', label='population',
         color='C1', edgecolor='black', alpha=0.5, density=True)
plt.hist(cat[cut]['D']/1000, bins, histtype='bar', label='sample',
         color='C0', edgecolor='black', alpha=0.5, density=True)
plt.title('Marginal Distance Distribution', fontsize=15)
plt.xlabel('Distance (kpc)', fontsize=14)
plt.xscale('log')
plt.legend()
plt.show()

In [ ]:
prefix = ''

# Create gbmcmc input file for selected binaries
gbmcmc_columns = ['f', 'fdot', 'colat', 'lon', 'A', 'incl', 'psi', 'phi']
gbmcmc = pd.DataFrame(columns=gbmcmc_columns)
for column in gbmcmc_columns:
    gbmcmc[column] = cat[cut][column]
gbmcmc.to_csv(f'{prefix}binaries.dat', float_format='%.9g',
              sep=' ', header=None, index=False)

# Create data file for selected binaries for comparison with gwemlisa data
output_columns = cat_columns + ['l', 'b', 'Mc', 'D']
output = pd.DataFrame(columns=output_columns)
for column in output_columns:
    output[column] = cat[cut][column]
output.to_csv(f'{prefix}gbfisher_parameters.dat', float_format='%.9g',
              sep=' ', header=output_columns, index=False)